In [1]:
import os
os.chdir('D:/Capstone/')

In [2]:
## Import Libraries here
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from numpy import array
import pandas as pd
import string
import os
import glob
from time import time
from keras import models
from keras.applications import densenet
import shutil
import re
import tensorflow as tf
import tensorflow_hub as hub
import pickle



Using TensorFlow backend.


In [7]:
## Variables
images_unzip_path = 'D:/Capstone/NLMCXR_png/'
reports_unzip_path = 'D:/Capstone/NLMCXR_reports/'
directory_in_str=reports_unzip_path+'/ecgen-radiology'
pathlist = Path(directory_in_str).glob('**/*.xml')
glove_dir = '/content/drive/My Drive/AIML/Capstone/Glove/'
# Below path contains all the images
images = images_unzip_path
train_images = images_unzip_path
proj_path = 'D:/Capstone/'
chexnet_weights = '/content/drive/My Drive/AIML/Capstone/brucechou1983_CheXNet_Keras_0.3.0_weights.h5'
para_pickle = proj_path+"para.pickle"
first_sent_pickle = proj_path+"first_sent.pickle"
word_vocab_pickle = proj_path+"word_vocab.pickle"

In [8]:
#Load the paragraph from findings
with open(para_pickle, 'rb') as f:
    para_findings = pickle.load(f)

In [9]:
#Load the first sentence from findings
with open(first_sent_pickle, 'rb') as f:
    first_sent_findings = pickle.load(f)

In [10]:
#Load the word vocab from findings
with open(word_vocab_pickle, 'rb') as f:
    word_vocab_findings = pickle.load(f)

In [14]:
# Create a list of all the training captions for Findings
all_para_findings = []
for key, val in para_findings.items():
    for cap in val:
        all_para_findings.append(cap)
print(len(all_para_findings))

all_first_sent_findings = []
for key, val in first_sent_findings.items():
    for cap in val:
        all_first_sent_findings.append(cap)
print(len(all_first_sent_findings))

24288
3668


In [15]:
# Build Word Vocab
word_count_threshold = 1
findings_word_counts = {}

nsents = 0
for sent in all_para_findings:
    nsents += 1
    for w in sent.split(' '):
        findings_word_counts[w] = findings_word_counts.get(w, 0) + 1

findings_w_vocab = [w for w in findings_word_counts if findings_word_counts[w] >= word_count_threshold]
print('preprocessed findings words %d -> %d' % (len(findings_word_counts), len(findings_w_vocab)))


preprocessed findings words 2145 -> 2145


In [24]:
ixtoword = {}
wordtoix = {}


ix = 1
for w in findings_w_vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [25]:
findings_vocab_size = len(ixtoword) + 1 # one for appended 0's
print(findings_vocab_size)

2146


In [27]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
findings_max_length = max_length(para_findings)
print('Findings Description Length: %d' % findings_max_length)

Findings Description Length: 50
